# Interim (Relational) Database Analysis Comparing BigFix Inventory and Webreports Data

## About

This report examines the BigFix Webreports and Inventory datasets to understand Database software at the agency.

In [ ]:
# import needed libs
import sys
sys.path.append("../lib/app_utilities")
from db_util import *
from IPython.display import display, HTML

In [ ]:
# make the connection to the databases we want to use for analysis
# CONN1 DB was constructed from BigFix Webreport data taken on April 4th, 2018
# CONN2 DB was constructed from BigFix Inventory data taken on April 17th, 2018
CONN1 = create_connection('apps2')
CONN2 = create_connection('bfi')

In [ ]:
# find install info for rdbs in BigFix Webreports ("Webreports")
wr_installs = count_rdbs_installs(CONN1)

In [ ]:
# find install info for rdbs in BigFix Inventory ("BFI")
bfi_installs = count_rdbs_installs(CONN2, table='device_software')

In [ ]:
# all of the db software where we have 1 or more installs (for any type/permutation)
data = pd.concat([wr_installs[wr_installs['num_installs'] > 0][:10], bfi_installs[bfi_installs['num_installs'] > 0][:10]], axis=1)

## Analysis

Here is a side by side comparison of the top 10 number of RDBS installs for Webreports (left) vs BFI (right) data.

In [ ]:
data.columns = ['wr installs', 'wr software', 'bfi installs', 'bfi software']
HTML(data.to_html())

Clearly these sources do not agree. For the remainder of the report we focus on the glaring discrepancy with the top item in each list (Microsoft SQL Server vs MS Access). 

### Microsoft SQL Server discrepancy analysis

We can resolve some of the problems comparing Webreports and BFI data for Microsoft SQL Server by spliting it into sub-categories. In addition to simply "Microsoft SQL Server" we can see that SQL Server has sub-types : "Compact", "R2", "LocalDB", "RC". *BFI and Webreports install data are approximately within 50% of each other with the exception of "Compact" which are wildly different (121 vs 3096). This is a significant difference which we cannot currently explain.*

### Microsoft Access discrepancy analysis

If we take a look at BFI types which match "Microsoft Access" we find the vast majority are related to Office 2013 installations, e.g.

In [ ]:
# Top 5 number of install types on BFI for Microsoft Access
bfi_access = db_software_histo("Microsoft Access%", conn=CONN2, table='device_software')[:5]
# Top 5 number of install types from WR for Microsoft Access
wr_access = db_software_histo("Microsoft Access%", conn=CONN1)[:5]

In [ ]:
# combine and present access information for both BFI and WR
data3 = pd.concat([wr_access, bfi_access], axis=1)
data3.columns = ['wr_software_name', 'wr_installs', 'bfi_software_name', 'bfi_installs']
data3

Investigation of the BFI signature file indicates that it is using the registry to find these extra instances by simply counting "Microsoft Office Professional 2013" installs (Quick online searches confirm that Office 2013 and 2016 Professional editions ship with Access DB). Here are some of the relevant rules:

    <sam__file_rule signature_guid="3955D6F1-9F5B-3C25-BF24-29EE8EA4605A" name="msaccess.exe" version="15.0"/>
    
    <sam__registry_rule signature_guid="3955D6F1-9F5B-3C25-BF24-29EE8EA4605A" key="HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Windows\CurrentVersion\Uninstall\*" value="DisplayNameVersion" expected="Microsoft Office Professional Plus 2013 - *"/>
    
Using this information and trying to search the Webreports data we find the top office installs to be (windows machines only): 

<table>
    <tr><th>software_name</th><th>num</th></tr>  
<tr><td>
 Microsoft Office Professional Plus 2013</td><td>
    19156</td></tr>
<tr><td>
 Microsoft Office Professional Plus 2016</td><td>
    18102</td></tr>
 <tr><td>
 Microsoft Office Professional Plus 2010 
       </td><td>
    502
         </td></tr>
<tr><td>
Microsoft Office Standard 2016  
      </td><td>
488
        </td></tr>
 <tr><td>
 Microsoft Office Standard 2013   
       </td><td>
433
         </td></tr>
 <tr><td>
 Microsoft Office Professional 2013 - en-us   
       </td><td>
 233
         </td></tr>
 <tr><td>
 Microsoft Office Visio Professional 2007   
       </td><td>
  227
         </td></tr>
 <tr><td>
 Microsoft Office Professional Plus 2007     
       </td><td>
 185
         </td></tr>
 <tr><td>
 Microsoft Office Home and Business 2013 - en-us  
       </td><td>
182
         </td></tr>
 <tr><td>
 Microsoft Office Standard 2010          
       </td><td>
 152
         </td></tr>
 <tr><td>
 Microsoft Office Standard 2007   
       </td><td>
 143
         </td></tr>
<tr><td>
Microsoft Office Home and Business 2016 - en-us   
      </td><td>
134
        </td></tr>
</table>

So indeed we see that we have lots of Office installs which qualify in Webreports data, but *too many, by a factor of 4* (9515 vs 19156 + 18102). The question remains as to why we recover more instances of office 2013 and 2016 from Webreports than BFI. 

## Summary

Clearly these datasources are not entirely consistent. We are aware that there are problems in the MacOS and (possibly) linux machine data. That said, the majority of the installs for SQL Server and Access are (probably) on windows machines so that is likely not much of an issue.

In both examined cases we see that Webreports is uncovering more instances than BFI, which is surprising. ***Looking more closely at Webreports data, we do see some duplicate machines matching --- need to investigate further here*** Possibly duplications of installs may be pushing Webreports numbers up, but this cannot be yet confirmed that this is the sole source of the discrepancy.

In summary neither datasource can be trusted at this point as we do not sufficiently understand the full nature of either datasource.

### Appendices

#### Suplementary Analysis Stuff

Table 1. Aligning BFI and Webreports installs by ID (note that "SQL Server LocalDB" is misaligned in the table below, it should match "Microsoft SQL Server LocalDB")

In [ ]:
data2 = wr_installs.merge(bfi_installs, left_on='software_name', right_on='software_name')
data2.columns = ['webreports', 'db name', 'bfi']
data2[data2['db name'].str.contains('SQL Server')]